 Looking to find the most expensive metro areas by median home price in the US.
 
Context

Zillow has a lot of data about housing prices in America.

Content

Data about housing prices and rental prices broken down according to city and state and number of bedrooms. More detail can be found at https://www.zillow.com/research/data/ and at https://www.zillow.com/research/home-sales-methodology-7733/.

Acknowledgements

The data was downloaded from https://www.zillow.com/research/data/. Banner photo from Ian Keefe on Unsplash. Dataset license described at https://www.zillow.com/research/data/.

In [1]:
#data organization and upload
import numpy as np
import pandas as pd

#visualizations
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

#setting filter warnings to ignore to keep our notebook clean
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv("City_Zhvi_2bedroom.csv")

In [3]:
data.head()

,Unnamed: 0,RegionID,RegionName,State,Metro,CountyName,SizeRank,2008-01,2008-02,2008-03,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
0,0,6181,New York,NY,New York-Newark-Jersey City,Queens County,1,576827.0,576551.0,575294.000000,...,755775.000000,755544.000000,755377.333333,755510.333333,754488.666667,753482.666667,751084.000000,747943.000000,745139.333333,743173.333333
1,1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,2,NaN,NaN,NaN,...,599677.666667,598162.333333,598823.000000,600231.000000,603184.333333,607904.666667,613860.000000,620111.666667,623957.333333,626501.666667
2,2,39051,Houston,TX,Houston-The Woodlands-Sugar Land,Harris County,3,97050.0,96793.5,96671.666667,...,135923.000000,136778.666667,137111.666667,137613.666667,138303.000000,138913.333333,139524.000000,139964.666667,140836.000000,140799.000000
3,3,17426,Chicago,IL,Chicago-Naperville-Elgin,Cook County,4,NaN,NaN,NaN,...,226506.333333,226949.666667,227951.000000,228515.333333,228289.000000,227479.666667,226601.333333,225887.333333,225615.666667,225706.000000
4,4,6915,San Antonio,TX,San Antonio-New Braunfels,Bexar County,5,80354.0,80612.0,80738.666667,...,116293.333333,117501.000000,118332.666667,118989.666667,119935.333333,120798.000000,121715.000000,122682.000000,122805.666667,122679.000000


In [4]:
columns = list(data.columns)
columns = columns[1:]

In [5]:
df =data[columns]
df.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,2008-01,2008-02,2008-03,2008-04,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
0,6181,New York,NY,New York-Newark-Jersey City,Queens County,1,576827.0,576551.0,575294.000000,573466.333333,...,755775.000000,755544.000000,755377.333333,755510.333333,754488.666667,753482.666667,751084.000000,747943.000000,745139.333333,743173.333333
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,2,NaN,NaN,NaN,NaN,...,599677.666667,598162.333333,598823.000000,600231.000000,603184.333333,607904.666667,613860.000000,620111.666667,623957.333333,626501.666667
2,39051,Houston,TX,Houston-The Woodlands-Sugar Land,Harris County,3,97050.0,96793.5,96671.666667,96087.000000,...,135923.000000,136778.666667,137111.666667,137613.666667,138303.000000,138913.333333,139524.000000,139964.666667,140836.000000,140799.000000
3,17426,Chicago,IL,Chicago-Naperville-Elgin,Cook County,4,NaN,NaN,NaN,213804.000000,...,226506.333333,226949.666667,227951.000000,228515.333333,228289.000000,227479.666667,226601.333333,225887.333333,225615.666667,225706.000000
4,6915,San Antonio,TX,San Antonio-New Braunfels,Bexar County,5,80354.0,80612.0,80738.666667,80994.333333,...,116293.333333,117501.000000,118332.666667,118989.666667,119935.333333,120798.000000,121715.000000,122682.000000,122805.666667,122679.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19552 entries, 0 to 19551
Columns: 149 entries, RegionID to 2019-11
dtypes: float64(143), int64(2), object(4)
memory usage: 22.2+ MB


In [7]:
len(df.RegionID.unique())

19552

In [8]:
len(df.Metro.unique())

850

Monthly data from Jan 2008 to November 2019.  142 months, 19,551 cities or towns and 850 metro areas

deal with nans - first in metro

In [9]:
metro_df = df.copy()
metro_df.fillna(0,inplace=True)

In [10]:
metro_df.iloc[:,:6].isna().sum()

RegionID      0
RegionName    0
State         0
Metro         0
CountyName    0
SizeRank      0
dtype: int64

In [11]:
print("{}% loss of data".format(round((4350/19551)*100,2)))

22.25% loss of data


ouch but some cities dont have metros believe it or not.  so Even though there is a big loss i think its the right move to drop



In [12]:
metro_df.iloc[:,:6].isna().sum()

RegionID      0
RegionName    0
State         0
Metro         0
CountyName    0
SizeRank      0
dtype: int64

In [13]:
metro_df.Metro[metro_df.Metro == 0].count()

4350

In [14]:
metro_df.drop(metro_df.index[metro_df.Metro ==0], inplace = True)


In [15]:
metro_df.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,2008-01,2008-02,2008-03,2008-04,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
0,6181,New York,NY,New York-Newark-Jersey City,Queens County,1,576827.0,576551.0,575294.000000,573466.333333,...,755775.000000,755544.000000,755377.333333,755510.333333,754488.666667,753482.666667,751084.000000,747943.000000,745139.333333,743173.333333
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,2,0.0,0.0,0.000000,0.000000,...,599677.666667,598162.333333,598823.000000,600231.000000,603184.333333,607904.666667,613860.000000,620111.666667,623957.333333,626501.666667
2,39051,Houston,TX,Houston-The Woodlands-Sugar Land,Harris County,3,97050.0,96793.5,96671.666667,96087.000000,...,135923.000000,136778.666667,137111.666667,137613.666667,138303.000000,138913.333333,139524.000000,139964.666667,140836.000000,140799.000000
3,17426,Chicago,IL,Chicago-Naperville-Elgin,Cook County,4,0.0,0.0,0.000000,213804.000000,...,226506.333333,226949.666667,227951.000000,228515.333333,228289.000000,227479.666667,226601.333333,225887.333333,225615.666667,225706.000000
4,6915,San Antonio,TX,San Antonio-New Braunfels,Bexar County,5,80354.0,80612.0,80738.666667,80994.333333,...,116293.333333,117501.000000,118332.666667,118989.666667,119935.333333,120798.000000,121715.000000,122682.000000,122805.666667,122679.000000


In [16]:
metro_df.iloc[:,:6].isna().sum()

RegionID      0
RegionName    0
State         0
Metro         0
CountyName    0
SizeRank      0
dtype: int64

In [17]:
metro_df = metro_df.iloc[:,1:]

In [18]:
metro_df.head()

,RegionName,State,Metro,CountyName,SizeRank,2008-01,2008-02,2008-03,2008-04,2008-05,...,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11
0,New York,NY,New York-Newark-Jersey City,Queens County,1,576827.0,576551.0,575294.000000,573466.333333,569521.0,...,755775.000000,755544.000000,755377.333333,755510.333333,754488.666667,753482.666667,751084.000000,747943.000000,745139.333333,743173.333333
1,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,2,0.0,0.0,0.000000,0.000000,0.0,...,599677.666667,598162.333333,598823.000000,600231.000000,603184.333333,607904.666667,613860.000000,620111.666667,623957.333333,626501.666667
2,Houston,TX,Houston-The Woodlands-Sugar Land,Harris County,3,97050.0,96793.5,96671.666667,96087.000000,95375.0,...,135923.000000,136778.666667,137111.666667,137613.666667,138303.000000,138913.333333,139524.000000,139964.666667,140836.000000,140799.000000
3,Chicago,IL,Chicago-Naperville-Elgin,Cook County,4,0.0,0.0,0.000000,213804.000000,215913.0,...,226506.333333,226949.666667,227951.000000,228515.333333,228289.000000,227479.666667,226601.333333,225887.333333,225615.666667,225706.000000
4,San Antonio,TX,San Antonio-New Braunfels,Bexar County,5,80354.0,80612.0,80738.666667,80994.333333,80945.0,...,116293.333333,117501.000000,118332.666667,118989.666667,119935.333333,120798.000000,121715.000000,122682.000000,122805.666667,122679.000000


In [19]:
df_prices = pd.DataFrame(columns =['region', 'state', 'metro','size_rank','mean'])

In [20]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 5 columns):
region       0 non-null object
state        0 non-null object
metro        0 non-null object
size_rank    0 non-null object
mean         0 non-null object
dtypes: object(5)
memory usage: 0.0+ bytes


In [21]:
df_prices.region = metro_df.RegionName
df_prices.state = metro_df.State
df_prices.metro = metro_df.Metro
df_prices.size_rank = metro_df.SizeRank

In [22]:
means = []
for i in range(len(df_prices.region)):
    val = np.mean(metro_df.iloc[i,6:])
    means.append(val)
df_prices['mean'] = means

In [23]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15202 entries, 0 to 19551
Data columns (total 5 columns):
region       15202 non-null object
state        15202 non-null object
metro        15202 non-null object
size_rank    15202 non-null int64
mean         15202 non-null float64
dtypes: float64(1), int64(1), object(3)
memory usage: 712.6+ KB


food for thought - use the mean or median of the median home prices.  

In [24]:
metros = list(df_prices.metro.unique())

In [29]:
metro_dict = {}
for msa in metros:        
    metro_dict[msa]= np.mean(df_prices[df_prices.metro==msa])

In [33]:
metro_dict['New York-Newark-Jersey City']

size_rank      5893.636511
mean         314474.951721
dtype: float64

find the top - plot circle for median price with city size as the other dimension?  plot scatter/line/bar? where each dot is a city and the height of the dot is the median price?  create a sorted list high to low? 

consider finding the median income for each msa and comparing that vs. median price to arrive at true affordability of home ownership